In [1]:
import pandas as pd
# from datetime import datetime

path_17 = "/Users/pcsishun/project_scg/project_hlr/data/data_2025-7-23.csv"
df_17 = pd.read_csv(path_17)


df = df_17
df = df.drop(columns=['id', 'temp', 'humidity', 'adjust_co2'])
df['timestamp'] = pd.to_datetime(df['timestamp'], utc=True)
df['timestamp_thai'] = df['timestamp'].dt.tz_convert('Asia/Bangkok')
df['time_only'] = df['timestamp_thai'].dt.time
# df
start_time = pd.to_datetime("08:45:00").time()
end_time = pd.to_datetime("11:46:00").time()

df['formatted'] = df['timestamp_thai'].dt.strftime('%Y/%m/%d %H:%M:%S')
df = df[(df['time_only'] >= start_time) & (df['time_only'] <= end_time)]

df = df[df['device_name'].isin(['tongdy_3', 'tongdy_4'])]
df['device_name_label'] = df['device_name_label'].replace({'Carbon inlet': 'CO2 HLR Inlet'})

pivot_df = df.pivot(index='formatted', columns=['device_name_label'], values='co2')
pivot_df = pivot_df.reset_index()
pivot_df.columns.name = None
pivot_df = pivot_df.rename(columns={
    'Carbon Outlet': 'CO2 HLR Inlet',
    'HLR Exhaust': 'CO2 HLR Exhaust'
})
pivot_df['formatted'] = pd.to_datetime(pivot_df['formatted'])
pivot_df['minute'] = pivot_df['formatted'].dt.floor('min')
mean_per_minute = pivot_df.groupby('minute').mean().reset_index()
mean_per_minute = mean_per_minute.rename(columns={'minute': 'timestamp'})
mean_per_minute = mean_per_minute.drop(columns=['formatted'])
mean_per_minute


,timestamp,CO2 HLR Inlet,CO2 HLR Exhaust
0,2025-07-23 08:45:00,430.0,470.5
1,2025-07-23 08:46:00,988.5,576.0
2,2025-07-23 08:47:00,1968.5,1035.5
3,2025-07-23 08:48:00,2270.5,1243.5
4,2025-07-23 08:49:00,2376.5,1288.0
...,...,...,...
176,2025-07-23 11:41:00,2665.5,2517.5
177,2025-07-23 11:42:00,2673.5,2523.5
178,2025-07-23 11:43:00,2676.5,2525.5
179,2025-07-23 11:44:00,2673.0,2525.0


In [2]:
mean_per_minute.to_csv('hlr_23_07_2025_0845-1145.csv')